# Lab 05 · Simple Agent and Tools

*This lab notebook provides guided steps. All commands are intended for local execution.*

## Objectives
- A minimal agent loop is expressed with limited iterations.
- Tool abstractions for calculator, db_query, and search_faq are prepared.
- Tool call timelines are logged for review.

## What will be learned
- Agent planning loops are reasoned about with deterministic stops.
- Tool registration strategies are documented.
- Logging of tool usage is practiced for observability.

## Prerequisites & install
The following commands are intended for local execution.

```bash
cd ai-web/backend
. .venv/bin/activate
pip install numpy
```

## Step-by-step tasks
### Step 1: Tool definitions
Lightweight tool functions are placed in a tools module.

In [ ]:
from pathlib import Path
module = Path("ai-web/backend/app/tools.py")
module.write_text('''from typing import Any, Dict, List


TOOL_LOG: List[Dict[str, Any]] = []


def calculator(expression: str) -> str:
    TOOL_LOG.append({"tool": "calculator", "input": expression})
    try:
        value = eval(expression, {"__builtins__": {}}, {})
    except Exception:
        return "A calculation error was observed."
    return str(value)


def db_query(sql: str) -> str:
    TOOL_LOG.append({"tool": "db_query", "input": sql})
    return "A mock database response was produced."


def search_faq(question: str) -> str:
    TOOL_LOG.append({"tool": "search_faq", "input": question})
    return "A documented FAQ entry was suggested."
''')
print("Tools module was created.")

### Step 2: Agent loop outline
A simple agent loop is introduced with a two-iteration limit.

In [ ]:
from pathlib import Path
agent_path = Path("ai-web/backend/app/agent.py")
agent_path.write_text('''from typing import Dict, List

from .tools import TOOL_LOG, calculator, db_query, search_faq


def run_agent(task: str) -> Dict[str, List[str]]:
    TOOL_LOG.clear()
    thoughts = [f"The task was received: {task}"]
    for step in range(2):
        if "calculate" in task and step == 0:
            result = calculator("1 + 1")
            thoughts.append(f"Calculator returned {result}.")
        elif "database" in task and step == 0:
            result = db_query("SELECT * FROM items LIMIT 1")
            thoughts.append(result)
        else:
            result = search_faq(task)
            thoughts.append(result)
    timeline = [f"{entry['tool']} ← {entry['input']}" for entry in TOOL_LOG]
    return {"thoughts": thoughts, "timeline": timeline}
''')
print("Agent loop was documented.")

### Step 3: Endpoint exposure
The agent run is exposed through FastAPI for easy testing.

In [ ]:
from pathlib import Path
main_path = Path("ai-web/backend/app/main.py")
text = main_path.read_text()
if "agent_endpoint" not in text:
    addition = '''
from .agent import run_agent


@app.post("/api/agent")
def agent_endpoint(payload: dict):
    task = payload.get("task", "A task was not specified.")
    result = run_agent(task)
    return result
'''
    main_path.write_text(text.rstrip() + "
" + addition)
    print("Agent endpoint was appended.")
else:
    print("Agent endpoint already present.")

## Validation / acceptance checks
```bash
# locally
curl -X POST http://localhost:8000/api/agent -H 'Content-Type: application/json' -d '{"task":"calculate 1+1"}'
```
- The response includes thoughts and a timeline reflecting tool usage.
- React development mode shows the described UI state without console errors.

## Homework / extensions
- Tool error handling pathways are drafted for robustness.
- Agent iteration limits are experimented with for longer plans.